In [1]:
'''
fine-tune CodeBERT
Author: Liu Jin Cheng
'''
from transformers import GPT2ForSequenceClassification,GPT2Tokenizer,get_linear_schedule_with_warmup
from datasets import load_dataset,Dataset
import os
import random
import numpy as np
import evaluate
import torch
import argparse
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.functional import F
from torch.cuda.amp import autocast as autocast,GradScaler
import pandas as pd
from sklearn.metrics import f1_score
from torch.optim import AdamW
import pandas as pd


/home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = os.path.join("..","preprocess","dataset")
train_batch_size = 4
eval_batch_size = 2
lr = 7e-5
num_epochs = 30
model_name = "../../gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name,bos_token = "<|startoftext|>",eos_token = "<|endoftext|>",pad_token = "<|pad|>",cls_token = "<|cls|>",sep_token = "<|sep|>" ,model_max_length = 1024)
model = GPT2ForSequenceClassification.from_pretrained(model_name,num_labels = 66)
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ../../gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50261, 768)

In [3]:

def compute_metrics(eval_pred):
        metirc = evaluate.load("accuracy")
        logits , labels = eval_pred
        predictions = np.argmax(logits,axis=-1)
        return metirc.compute(predictions=predictions,references=labels)


def tokenize_function(examples):
        return tokenizer(examples["text"],truncation = True,padding=True) 

def collate_fn(examples):
    return tokenizer.pad(examples, padding="max_length", return_tensors="pt")

In [4]:
dftrain = pd.read_pickle(os.path.join(data_path,"train.pkl"))
dfvalid = pd.read_pickle(os.path.join(data_path,"valid.pkl"))
traindatasets = Dataset.from_pandas(dftrain)
validdatasets = Dataset.from_pandas(dfvalid)

In [5]:
train_tokenized_dataset = traindatasets.map(tokenize_function,batched=True,remove_columns=["text","__index_level_0__"]).rename_column("label","labels")
valid_tokenized_dataset = validdatasets.map(tokenize_function,batched=True,remove_columns=["text","__index_level_0__"]).rename_column("label","labels")
train_dataloader = DataLoader(train_tokenized_dataset,shuffle=True,collate_fn=collate_fn,batch_size = train_batch_size)
eval_dataloader = DataLoader(valid_tokenized_dataset , collate_fn=collate_fn,batch_size = eval_batch_size)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=66, bias=False)
)

In [6]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)


In [7]:
model.to(device)
max_eval_acc = 0
iter_to_accumlate = 4
epochloss = []
trainlogdf = pd.DataFrame(columns=["step","trainloss","validloss","acc","f1-score"])
rowindex = 0
eval_no_progress_count = 0
early_stopping = 6
early_stopping_flag = False
for epoch in range(num_epochs):
    model.train()
    allloss = 0
    for step,batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss/iter_to_accumlate
        loss.backward()
        allloss += loss.item()
        trainlogdf.loc[rowindex] = [rowindex,loss.item(),None,None,None]
        rowindex += 1
        epochloss.append(loss.item())
        if (step+1)%iter_to_accumlate==0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        if (step+1)%(50*iter_to_accumlate) == 0:
            print("epoch",epoch,"step",step,"loss",loss,sep=" ")
        
        if (step+1)%800 == 0:
            count = 0
            model.eval()
            validloss = 0
            preds = []
            labels = []
            for evalstep,batch in enumerate(tqdm(eval_dataloader)):
                labels += batch['labels'].cpu()
                batch.to(device)
                with torch.no_grad():
                    output = model(**batch)
                validloss += output.loss.item()
                pred = torch.argmax(F.softmax(output.logits.cpu(),dim=1),dim=1)
                preds += pred
                count += int(sum(batch['labels'].cpu() == pred))
            model.train()
            eval_acc = count/2732
            trainlogdf.loc[rowindex-1,"validloss"] = validloss/2732
            trainlogdf.loc[rowindex-1,"acc"] = eval_acc
            trainlogdf.loc[rowindex-1,"f1-score"] = f1_score(np.array(labels),np.array(preds),average="macro")
            print("epoch ",epoch,"step",step,"acc ",eval_acc)
            if eval_acc < max_eval_acc:
                eval_no_progress_count += 1
                if eval_no_progress_count >=early_stopping:
                    print("Early Stopping:Epoch",epoch," Step",step,"Eval_acc",eval_acc,sep=" ")
                    early_stopping_flag = True
                    break
                else:
                    print("Early Stopping record count",eval_no_progress_count,"/",early_stopping,"Will stop","Max eval acc",max_eval_acc,sep=" ")
            if eval_acc > max_eval_acc:
                max_eval_acc = eval_acc
                print("Update Max eval acc",max_eval_acc)
                eval_no_progress_count = 0
                model.save_pretrained("GPT2saved_models")
                torch.save(model.state_dict(),os.path.join("checkpoint","model.bin"))
                torch.save(optimizer.state_dict(),os.path.join("checkpoint","optimizer.bin"))
                torch.save(lr_scheduler.state_dict(),os.path.join("checkpoint","lr_scheduler.bin"))
    print("epoch",epoch,"step",step,"trainLoss:",allloss/(len(train_dataloader)*train_batch_size))
    if early_stopping_flag:
        break
trainlogdf.to_csv("trainlog.csv")
tokenizer.save_pretrained("GPT2saved_models")

  4%|▎         | 200/5464 [01:55<49:07,  1.79it/s]

epoch 0 step 199 loss tensor(3.0066, device='cuda:0', grad_fn=<DivBackward0>)


  7%|▋         | 400/5464 [03:50<49:02,  1.72it/s]

epoch 0 step 399 loss tensor(2.0652, device='cuda:0', grad_fn=<DivBackward0>)


 11%|█         | 600/5464 [05:45<46:49,  1.73it/s]

...

 44%|████▍     | 2399/5464 [12:40<13:19,  3.83it/s]

epoch 7 step 2399 loss tensor(0.0580, device='cuda:0', grad_fn=<DivBackward0>)


 44%|████▍     | 2399/5464 [13:46<17:36,  2.90it/s]


epoch  7 step 2399 acc  0.6346998535871157
Early Stopping:Epoch 7  Step 2399 Eval_acc 0.6346998535871157
epoch 7 step 2399 trainLoss: 0.0134672060180099


('GPT2saved_models/tokenizer_config.json',
 'GPT2saved_models/special_tokens_map.json',
 'GPT2saved_models/vocab.json',
 'GPT2saved_models/merges.txt',
 'GPT2saved_models/added_tokens.json')

In [8]:
!source attack.sh

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/ljc/desktop/neutral-attack-for-pretrained-models/GPT2/Defect-detection │
│ /code/attack.py:195 in <module>                                              │
│                                                                              │
│   192                                                                        │
│   193                                                                        │
│   194 if __name__ == '__main__':                                             │
│ ❱ 195 │   main()                                                             │
│   196                                                                        │
│                                                                              │
│ /home/ljc/desktop/neutral-attack-for-pretrained-models/GPT2/Defect-detection │
│ /code/attack.py:145 in main                                                  │
│                           